In [2]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
import pandas as pd
import os

In [54]:
# Define the path to the CSV file
csv_file_path = r'E:\DS440proj\BTCpred\volume\data\raw\Macro\raw_data.csv'

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Display the first few rows of the dataframe
print(df.head())

                    Datetime  SP500  DOW30  EuroStoxx50  NASDAQ  CrudeOil  \
0  2022-02-09 09:30:00+08:00    NaN    NaN          NaN     NaN       NaN   
1  2022-02-09 10:30:00+08:00    NaN    NaN          NaN     NaN       NaN   
2  2022-02-09 11:30:00+08:00    NaN    NaN          NaN     NaN       NaN   
3  2022-02-09 12:30:00+08:00    NaN    NaN          NaN     NaN       NaN   
4  2022-02-09 13:30:00+08:00    NaN    NaN          NaN     NaN       NaN   

           SSE  Gold  VIX  Nikkei225  FTSE100  EURUSD  CNYUSD  GBPUSD  JPYUSD  \
0  3465.004883   NaN  NaN        NaN      NaN     NaN     NaN     NaN     NaN   
1  3467.037354   NaN  NaN        NaN      NaN     NaN     NaN     NaN     NaN   
2  3466.365234   NaN  NaN        NaN      NaN     NaN     NaN     NaN     NaN   
3  3479.272461   NaN  NaN        NaN      NaN     NaN     NaN     NaN     NaN   
4  3480.633789   NaN  NaN        NaN      NaN     NaN     NaN     NaN     NaN   

   CHFUSD  
0     NaN  
1     NaN  
2     NaN  
3 

In [56]:
def fill_single_interval_missings(dataframe, columns):
    for column in columns:
        # Loop through each row except the first and last
        for i in range(1, len(dataframe) - 1):
            # Check if the current value is NaN and neighbors are not
            if pd.isna(dataframe.at[dataframe.index[i], column]) and \
               not pd.isna(dataframe.at[dataframe.index[i - 1], column]) and \
               not pd.isna(dataframe.at[dataframe.index[i + 1], column]):
                # Calculate the midpoint of its neighbors
                midpoint = (dataframe.at[dataframe.index[i - 1], column] + dataframe.at[dataframe.index[i + 1], column]) / 2
                # Replace the NaN with the midpoint
                dataframe.at[dataframe.index[i], column] = midpoint
    return dataframe

# Replace single-interval missing values for specified columns
df = fill_single_interval_missings(df, ['SP500', 'DOW30', 'NASDAQ','SSE'])


In [58]:
df

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
0,2022-02-09 09:30:00+08:00,NaN,NaN,NaN,NaN,NaN,3465.004883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-09 10:30:00+08:00,NaN,NaN,NaN,NaN,NaN,3467.037354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-02-09 11:30:00+08:00,NaN,NaN,NaN,NaN,NaN,3466.365234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-02-09 12:30:00+08:00,NaN,NaN,NaN,NaN,NaN,3479.272461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-02-09 13:30:00+08:00,NaN,NaN,NaN,NaN,NaN,3480.633789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18969,2024-02-12 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.077470,0.139036,1.262834,0.006697,1.142348
18970,2024-02-12 18:00:00-05:00,NaN,NaN,NaN,NaN,76.980003,NaN,2032.5,NaN,NaN,NaN,1.077470,0.139036,1.262690,0.006693,1.141735
18971,2024-02-12 19:00:00-05:00,NaN,NaN,NaN,NaN,76.940002,NaN,2033.0,NaN,37517.210938,NaN,1.077586,NaN,1.262563,0.006698,1.141800
18972,2024-02-12 20:00:00-05:00,NaN,NaN,NaN,NaN,77.019997,NaN,2033.5,NaN,37636.980469,NaN,1.077006,0.139036,1.262100,0.006695,1.141448


In [59]:
from pytz import timezone

# Convert 'Datetime' to datetime object and set timezone to UTC since original times are in +08:00
df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)

# Convert 'Datetime' from UTC to Eastern Time (considering Eastern Standard Time without daylight saving)
df['Datetime'] = df['Datetime'].dt.tz_convert('EST')

# Remove rows with half-hour times
df = df[~df['Datetime'].dt.minute.isin([30])]

# Display the first few rows of the modified dataframe
df.head()

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
18,2022-02-13 18:00:00-05:00,NaN,NaN,NaN,NaN,94.379997,NaN,1861.099976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2022-02-13 19:00:00-05:00,NaN,NaN,NaN,NaN,94.540001,NaN,1859.599976,NaN,27128.310547,NaN,1.136622,0.157381,1.356337,0.008666,1.081432
20,2022-02-13 20:00:00-05:00,NaN,NaN,NaN,NaN,94.610001,NaN,1861.099976,NaN,26986.789062,NaN,1.135718,0.157230,1.355528,0.008671,1.081326
22,2022-02-13 21:00:00-05:00,NaN,NaN,NaN,NaN,94.680000,3442.088135,1856.000000,NaN,26970.339844,NaN,1.134687,0.157252,1.354224,0.008660,1.080555
24,2022-02-13 22:00:00-05:00,NaN,NaN,NaN,NaN,94.440002,3441.650024,1854.300049,NaN,27118.580078,NaN,1.134816,0.157250,1.353693,0.008658,1.080976


In [60]:
data_filled = df.fillna(method='ffill')

# Display the first few rows of the dataframe to confirm changes
data_filled.head()

C:\Users\13275\AppData\Local\Temp\ipykernel_46268\1185419078.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_filled = df.fillna(method='ffill')


,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
18,2022-02-13 18:00:00-05:00,NaN,NaN,NaN,NaN,94.379997,NaN,1861.099976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2022-02-13 19:00:00-05:00,NaN,NaN,NaN,NaN,94.540001,NaN,1859.599976,NaN,27128.310547,NaN,1.136622,0.157381,1.356337,0.008666,1.081432
20,2022-02-13 20:00:00-05:00,NaN,NaN,NaN,NaN,94.610001,NaN,1861.099976,NaN,26986.789062,NaN,1.135718,0.157230,1.355528,0.008671,1.081326
22,2022-02-13 21:00:00-05:00,NaN,NaN,NaN,NaN,94.680000,3442.088135,1856.000000,NaN,26970.339844,NaN,1.134687,0.157252,1.354224,0.008660,1.080555
24,2022-02-13 22:00:00-05:00,NaN,NaN,NaN,NaN,94.440002,3441.650024,1854.300049,NaN,27118.580078,NaN,1.134816,0.157250,1.353693,0.008658,1.080976


In [61]:
data_filled

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
18,2022-02-13 18:00:00-05:00,NaN,NaN,NaN,NaN,94.379997,NaN,1861.099976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2022-02-13 19:00:00-05:00,NaN,NaN,NaN,NaN,94.540001,NaN,1859.599976,NaN,27128.310547,NaN,1.136622,0.157381,1.356337,0.008666,1.081432
20,2022-02-13 20:00:00-05:00,NaN,NaN,NaN,NaN,94.610001,NaN,1861.099976,NaN,26986.789062,NaN,1.135718,0.157230,1.355528,0.008671,1.081326
22,2022-02-13 21:00:00-05:00,NaN,NaN,NaN,NaN,94.680000,3442.088135,1856.000000,NaN,26970.339844,NaN,1.134687,0.157252,1.354224,0.008660,1.080555
24,2022-02-13 22:00:00-05:00,NaN,NaN,NaN,NaN,94.440002,3441.650024,1854.300049,NaN,27118.580078,NaN,1.134816,0.157250,1.353693,0.008658,1.080976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18969,2024-02-12 17:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,76.940002,2858.799683,2033.800049,13.93,36942.101562,7573.689941,1.077470,0.139036,1.262834,0.006697,1.142348
18970,2024-02-12 18:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,76.980003,2858.799683,2032.500000,13.93,36942.101562,7573.689941,1.077470,0.139036,1.262690,0.006693,1.141735
18971,2024-02-12 19:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,76.940002,2858.799683,2033.000000,13.93,37517.210938,7573.689941,1.077586,0.139036,1.262563,0.006698,1.141800
18972,2024-02-12 20:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,77.019997,2858.799683,2033.500000,13.93,37636.980469,7573.689941,1.077006,0.139036,1.262100,0.006695,1.141448


In [62]:
df = data_filled.dropna()

In [63]:
df

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
41,2022-02-14 10:00:00-05:00,4405.064941,34487.544922,4047.379883,13863.107422,93.570000,3425.213013,1870.000000,29.43,27059.099609,7507.370117,1.130966,0.157314,1.351205,0.008652,1.079774
43,2022-02-14 11:00:00-05:00,4414.295166,34579.794922,4057.600098,13902.955078,93.559998,3425.213013,1863.400024,29.27,27059.099609,7525.779785,1.130966,0.157314,1.351644,0.008641,1.079261
45,2022-02-14 12:00:00-05:00,4414.995117,34630.660156,4057.600098,13867.772949,94.019997,3425.213013,1866.300049,28.77,27059.099609,7525.779785,1.131350,0.157314,1.353125,0.008648,1.080520
47,2022-02-14 13:00:00-05:00,4398.870117,34520.121094,4057.600098,13798.939453,95.059998,3425.213013,1873.000000,29.34,27059.099609,7525.779785,1.130327,0.157314,1.352631,0.008663,1.080847
49,2022-02-14 14:00:00-05:00,4391.920166,34468.455078,4057.600098,13775.538086,94.760002,3425.213013,1869.900024,31.27,27059.099609,7525.779785,1.129943,0.157314,1.352357,0.008646,1.080264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18969,2024-02-12 17:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,76.940002,2858.799683,2033.800049,13.93,36942.101562,7573.689941,1.077470,0.139036,1.262834,0.006697,1.142348
18970,2024-02-12 18:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,76.980003,2858.799683,2032.500000,13.93,36942.101562,7573.689941,1.077470,0.139036,1.262690,0.006693,1.141735
18971,2024-02-12 19:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,76.940002,2858.799683,2033.000000,13.93,37517.210938,7573.689941,1.077586,0.139036,1.262563,0.006698,1.141800
18972,2024-02-12 20:00:00-05:00,5022.459961,38785.763672,4746.350098,15950.163086,77.019997,2858.799683,2033.500000,13.93,37636.980469,7573.689941,1.077006,0.139036,1.262100,0.006695,1.141448


In [64]:
df.to_csv("processed_macro_data.csv")